In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet
/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv
/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv
/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet
/kaggle/input/zzzs-lightweight-training-dataset-target/Zzzs_train.parquet
/kaggle/input/zzzs-lightweight-training-dataset-target/Zzzs_train_multi.parquet


* [EDA](#chapter1)
    * [Imports](#subsection1)
* [Feature Engineering](#chapter2)
    * [Section 3.1](#section_3_1)

# EDA <a class="anchor"  id="chapter1"></a>

## Imports <a class="anchor"  id="subsection1"></a>

In [2]:
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
import plotly.express as px

from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import RobustScaler
from IPython.display import display
from pandas.api.types import CategoricalDtype
from category_encoders import MEstimateEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from scipy import stats
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDOneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from matplotlib.ticker import MaxNLocator
from functools import partial

from sklearn.inspection import permutation_importance
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.metrics import mean_absolute_error, mean_squared_error

from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn import preprocessing
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from itertools import groupby
import gc
from time import time

import dask
import dask.array as da
import dask.dataframe as dd
import polars as pl

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

## Config

In [3]:
TARGET = 'awake'
RESULT_BASELINE = []

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

def plot_maxdeph_score(rd_search, randomizedSearchCV_model):
    max_depth = [item['model__max_depth'] for item in rd_search.named_steps[randomizedSearchCV_model].cv_results_['params']]
    scores = list(rd_search.named_steps[randomizedSearchCV_model].cv_results_['mean_test_score'])
    d = pd.DataFrame([max_depth, scores]).T
    d.columns = ['Max Depth','Score']
    d.groupby(['Max Depth']).mean()
    return d

# Mute warnings
warnings.filterwarnings('ignore')

/tmp/ipykernel_20/1232833359.py:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-whitegrid")


## Load data

In [4]:
# binary class data
# train = pd.read_parquet("/kaggle/input/zzzs-lightweight-training-dataset-target/Zzzs_train.parquet")
# multiclass data

# train = pd.read_parquet("/kaggle/input/zzzs-lightweight-training-dataset-target/Zzzs_train_multi.parquet")
# test  = pd.read_parquet("/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet")

train = (pl.scan_parquet("/kaggle/input/zzzs-lightweight-training-dataset-target/Zzzs_train_multi.parquet")
          .with_columns(
              (pl.col("timestamp")
                 .str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z")
                 .dt.hour()
                 .alias("hour")
              )
          )
          .drop("timestamp")
          .collect()
          .to_pandas()
)

# train = dd.read_parquet("/kaggle/input/zzzs-lightweight-training-dataset-target/Zzzs_train_multi.parquet")
# test  = dd.read_parquet("/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet")

## Data Statistics

In [5]:
train.head()

,series_id,step,anglez,enmo,awake,hour
0,08db4255286f,0,-30.845301,0.0447,1,10
1,08db4255286f,1,-34.181801,0.0443,1,10
2,08db4255286f,2,-33.877102,0.0483,1,10
3,08db4255286f,3,-34.282101,0.0680,1,10
4,08db4255286f,4,-34.385799,0.0768,1,10


In [6]:
train.describe()

,step,anglez,enmo,awake,hour
count,1.628982e+07,1.628982e+07,1.628982e+07,1.628982e+07,1.628982e+07
mean,2.023739e+05,-7.815899e+00,4.140666e-02,9.152462e-01,1.151986e+01
std,1.252966e+05,3.790032e+01,1.051808e-01,6.790839e-01,6.935843e+00
min,0.000000e+00,-9.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00
25%,9.608000e+04,-3.539180e+01,1.300000e-03,0.000000e+00,5.000000e+00
50%,1.954140e+05,-7.391900e+00,1.640000e-02,1.000000e+00,1.200000e+01
75%,3.011760e+05,1.595410e+01,3.970000e-02,1.000000e+00,1.800000e+01
max,6.346790e+05,9.000000e+01,7.016100e+00,2.000000e+00,2.300000e+01


In [7]:
[train.shape]

[(16289820, 6)]

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16289820 entries, 0 to 16289819
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   series_id  object 
 1   step       uint32 
 2   anglez     float32
 3   enmo       float32
 4   awake      int64  
 5   hour       uint32 
dtypes: float32(2), int64(1), object(1), uint32(2)
memory usage: 497.1+ MB


In [9]:
num_col = train.select_dtypes(exclude=['object'])#.drop([TARGET], axis=1).copy()
cat_col = train.select_dtypes(include=['object'])
ord_col = []
all_col = [f for f in train.columns if f != TARGET]

In [10]:
print(num_col.columns)
print(cat_col.columns)

Index(['step', 'anglez', 'enmo', 'awake', 'hour'], dtype='object')
Index(['series_id'], dtype='object')


## Understand data

## Univariate analysis

### Numeric features

For numerical features, we are always concerned about the distribution of these features, including the statistical characteristics of these columns e.g mean, median, mode. Hence we will usually use Distribution plot to visualize their data distribution. Boxplots are also commonly used to unearth the statistical characteristics of each feature. More often than not, we use it to look for any outliers that we might need to filter out later on during the preprocessing step.

In [11]:
# disc_num_var = ['id']

# cont_num_var = []
# for i in num_col.columns:
#     if i not in disc_num_var:
#         cont_num_var.append(i)

# fig = plt.figure(figsize=(16,35))
# for index,col in enumerate(cont_num_var):
#     plt.subplot(10,4,index+1)
#     sns.distplot(num_col.loc[:,col].dropna(), kde=False)
# fig.tight_layout(pad=2.0)

This plot help us idenytify some variables with mostly 1 value as seen from the plots. Features with mostly 0s are highly *skewed*. Having alot of 0s in the distribution doesnt really add information for predicting problems. Hence, we can remove them during our preprocessing step.

### Boxplot to identify outliers

In [12]:
# fig = plt.figure(figsize=(16,35))
# for index,col in enumerate(cont_num_var):
#     plt.subplot(10,4,index+1)
#     sns.boxplot(y=col, data=num_col.dropna())
# fig.tight_layout(pad=2.0)

In [13]:
# fig = plt.figure(figsize=(20,15))
# for index,col in enumerate(disc_num_var):
#     plt.subplot(10,3,index+1)
#     sns.countplot(x=col, data=num_col.dropna())
# fig.tight_layout(pad=1.0)

### Categorical features

In the case of categorical features, we will often use countplots to visualize the count of each distinct value within each features. We could find some categorical features that consist of mainly just one value, which does not add any useful information. Thus, we should also remove them later on.

In [14]:
# fig = plt.figure(figsize=(18,20))
# for index in range(len(cat_col.columns)):
#     plt.subplot(9,5,index+1)
#     sns.countplot(x=cat_col.iloc[:,index], data=cat_col.dropna())
#     plt.xticks(rotation=90)
# fig.tight_layout(pad=2.0)

Univariate Analysis helps us to understand all the features better, on an individual scale. To further deepen our insights and uncover potential pattern in the data, we will also need to find out more about the relationship between all these features with one another, which brings us to our next step in our analysis - Bivariate Analysis

## Bi-Variate Analysis

Bi-variate analysis looks at 2 different features to identify any possible relationship or distinctive patterns between the 2 features. One of the commonly used technique is through the Correlation Matrix. Correlation matrix is an effective tool to uncover linear relationship (Correlation) between any 2 continuous features. Correlation not only allow us to determine which features are important to TARGET, but also as a mean to investigate any multicollinearity between our independent predictors.
Multicollinearity happens when 2 or more independent variables are highly correlated with one another. In such situation, it causes precision loss in our regression coefficients, affecting our ability to identify the most important features that are most useful to our model

### Correlation matrix

In [15]:
# plt.figure(figsize=(14,12))
# correlation = num_col.corr()
# sns.heatmap(correlation, mask = correlation <0.8, linewidth=0.5, cmap='Blues')

From the correlation matrix we didnt identify any variables which are highly correlated with each other. Otherwise it would guide us in our preprocessing steps later on as we aim to remove highly correlated features to avoid performance loss in our model

#### Identifying relationship between Numerical Predictor and Target

Below, we sorted the strength of linear relationship between TARGET and other features. Hence, these features will be important factor.

In [16]:
# num_col_TARGET = train.select_dtypes(exclude=['object'])
# correlation = num_col_TARGET.corr()
# correlation[[TARGET]].sort_values([TARGET], ascending=False)

#### Scatterplot

Using scatterplot can also help us to identify potential linear relationship between Numerical features. Although scatterplot does not provide quantitative evidence on the strength of linear relationship between our features, it is useful in helping us to visualize any sort of relationship that correlation matrix could not calculate. E.g Quadratic, Exponential relationships.

In [17]:
# fig = plt.figure(figsize=(16,35))
# for index in range(len(train.columns)):
#     plt.subplot(10,4,index+1)
#     sns.scatterplot(x=train.iloc[:,index], y=TARGET, data=train.dropna())
# fig.tight_layout(pad=2.0)

### Mutual information

Mutual information describes relationships in terms of uncertainty. The mutual information between two quantities is a measure of the extent to which knowledge of one quantity reduces uncertainty about the other. If you knew the value of a feature, how much more confident would you be about the target?

Mutual information is a lot like correlation in that it measures a relationship between two quantities. The advantage of mutual information is that it can detect any kind of relationship, while correlation only detects linear relationships. Mutual information can capture any kind of association (not just linear, like correlation.)

The least possible mutual information between quantities is 0.0. When MI is zero, the quantities are independent: neither can tell you anything about the other. Conversely, in theory there's no upper bound to what MI can be. In practice though values above 2.0 or so are uncommon. (Mutual information is a logarithmic quantity, so it increases very slowly.)

Here are some things to remember when applying mutual information:

* MI can help you to understand the relative potential of a feature as a predictor of the target, considered by itself.
* It's possible for a feature to be very informative when interacting with other features, but not so informative all alone. MI can't detect interactions between features. It is a univariate metric.
* The actual usefulness of a feature depends on the model you use it with. A feature is only useful to the extent that its relationship with the target is one your model can learn. Just because a feature has a high MI score doesn't mean your model will be able to do anything with that information. You may need to transform the feature first to expose the association.

In [18]:
def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    discrete_features = [pd.api.types.is_float_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

In [19]:
train.shape

(16289820, 6)

In [20]:
# X_raw = train.fillna(method='ffill')
# features = ['', '', '']
# sns.relplot(
#     x="value", y=TARGET, col="variable", data=X_raw.melt(id_vars=TARGET, value_vars=features), facet_kws=dict(sharex=False),
# );

Based on the plots, which feature do you think would have the highest mutual information with *TARGET*?
The highest mutual information with *TARGET* tends to constrain it to a smaller range of possible values.

The dataset may have a lot of features to work with all at once! Fortunately, you can identify the features with the most potential.
We can compute mutual information scores for the dataset features.

In [21]:
# X_mi = X_raw.copy()
# y_mi = X_mi.pop(TARGET)

In [22]:
# [X_mi.shape, y_mi.shape]

In [23]:
# mi_scores = make_mi_scores(X_mi, y_mi)

In [24]:
# print(mi_scores.head(20))
# #print(mi_scores.tail(20))  # uncomment to see bottom 20

# plt.figure(dpi=100, figsize=(8, 5))
# plot_mi_scores(mi_scores.head(20))
# #plot_mi_scores(mi_scores.tail(20))  # uncomment to see bottom 20

Do the scores seem reasonable? Do the high scoring features represent things you'd think most people would value in a home? Do you notice any themes in what they describe?

It's good that our mutual information metric scored features highly. On the other hand, the lowest ranked features seem to mostly represent things that are rare or exceptional in some way, and so wouldn't be relevant.

In [25]:
# sns.catplot(x=" ", y=TARGET, data=train, kind="boxen");

In [26]:
# feature = " "

# sns.lmplot(
#     x=feature, y=TARGET, hue="O2_1", col="O2_1",
#     data=train, scatter_kws={"edgecolor": 'w'}, col_wrap=3, height=4,
# );

### Check the distribution of target variable

In [27]:
# check the distribution of the target variable
train[TARGET].value_counts()

1    8660676
0    4504884
2    3124260
Name: awake, dtype: int64

In [28]:
#train

In [29]:
#train['timestamp'] = pd.to_datetime(train['timestamp']).apply(lambda t: t.tz_localize(None))
#train['timestamp'] = pd.to_datetime(train["timestamp"],utc=True)

## Data Processing

## Fine-Tune model

In [30]:
# X = train
# y = train[TARGET]

In [31]:
# X.shape

In [32]:
# # just for baseline
# features_num = X.select_dtypes(exclude=['object']).copy().columns
# features_cat = X.select_dtypes(include=['object']).copy().columns

# transformer_num = make_pipeline(
#     SimpleImputer(strategy="constant"), # there are a few missing values
#     #StandardScaler(),
#     RobustScaler()
# )
# transformer_cat = make_pipeline(
#     SimpleImputer(strategy="constant", fill_value="NA"),
#     OneHotEncoder(handle_unknown='ignore'),
# )

# preprocessor = make_column_transformer(
#     (transformer_num, features_num),
#     (transformer_cat, features_cat),
# )

In [33]:
#X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

### XGBoost

In [34]:
def XGBModel(X_train, y_train, preprocessor):    
    
#     param_lst_xgb = {
#         'learning_rate' : [0.01, 0.03, 0.1, 0.15, 0.3, 0.5],
#         'n_estimators' : [2, 10, 100],#500, 1000, 2000, 3000, 4000, 7000, 10000, 15000],
#         'max_depth' : [3, 6, 9],
#         'min_child_weight' : [1, 5, 10, 20],
#         'reg_alpha' : [0.001, 0.01, 0.1],
#         'reg_lambda' : [0.001, 0.01, 0.1]
#     }
    
#     xgb_dflt = XGBClassifier()

#     xgb_reg = RandomizedSearchCV(
#                                     estimator = xgb_dflt, 
#                                     param_distributions = param_lst_xgb,
#                                     n_iter = 2, 
#                                     scoring = 'accuracy',
#                                     n_jobs = 100,
#                                     cv = 2
#                                 )

    # Bundle preprocessing and modeling code in a pipeline
#     xgb_pipeline = Pipeline(steps=[
#                                     ('preprocessor', preprocessor),
#                                     ('model_xgb_RandomizedSearchCV', xgb_reg)
#                                  ])

#     xgb_search = xgb_pipeline.fit(X_train, y_train)

    # XGB with tune hyperparameters
#     xgb = XGBClassifier(**xgb_search.named_steps['model_xgb_RandomizedSearchCV'].best_params_)

    xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.01,
       max_delta_step=0, max_depth=9, min_child_weight=1, missing=1,
       n_estimators=100, n_jobs=100, nthread=None,
       objective='multi:softprob', random_state=0, reg_alpha=0.01,
       reg_lambda=0.01, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)
    
    # Bundle preprocessing and modeling code in a pipeline
    xgb_pipeline_best_params = Pipeline(steps=[
#                                     ('preprocessor', preprocessor),
                                    ('model', xgb)
                                 ])

    xgb_pipeline_best_params.fit(X_train, y_train)

#     print("best_params_ ", xgb_search.named_steps['model_xgb_RandomizedSearchCV'].best_params_)
#     print("best_score_ ", xgb_search.named_steps['model_xgb_RandomizedSearchCV'].best_score_)
#     print("best_estimator_ ", xgb_search.named_steps['model_xgb_RandomizedSearchCV'].best_estimator_)
#     print("cv_results_ ", xgb_search.named_steps['model_xgb_RandomizedSearchCV'].cv_results_)
#     print("mean_test_score ", xgb_search.named_steps['model_xgb_RandomizedSearchCV'].cv_results_['mean_test_score'])

    return xgb_pipeline_best_params

#xgb_pipeline = XGBModel(X_train, y_train, preprocessor)

### LightGBM

In [35]:
def LGBMModel(X_train, y_train, preprocessor):
    
    lgbm_dflt = LGBMClassifier()

    param_lst_lgbm = {
        'max_depth' : [2, 5, 8, 10],
        'learning_rate' : [0.001, 0.01, 0.03, 0.1, 0.2],
        'n_estimators' : [10],#, 300, 500, 1000, 1500, 4000, 7000, 10000, 15000],
        'lambda_l1' : [0.0001, 0.001, 0.01],
        'lambda_l2' : [0, 0.0001, 0.001, 0.01],
        'feature_fraction' : [0.4, 0.6, 0.8],
        'min_child_samples' : [5, 10, 20, 25]
    }

    lightgbm = RandomizedSearchCV(estimator = lgbm_dflt, 
                                  param_distributions = param_lst_lgbm,
                                  n_iter = 3, 
                                  scoring = 'accuracy',
                                  n_jobs = 20,
                                  cv = 5)

    lgbm_pipeline = Pipeline(steps=[
                                    ('preprocessor', preprocessor),
                                    ('model_lightgbm_RandomizedSearchCV', lightgbm)
                                 ])

    lightgbm_search = lgbm_pipeline.fit(X_train, y_train)

    #LightBGM with tuned hyperparameters
    lgbm = LGBMClassifier(**lightgbm_search.named_steps['model_lightgbm_RandomizedSearchCV'].best_params_, verbose_eval=False)

    lgbm_pipeline_best_params = Pipeline(steps=[
                                    ('preprocessor', preprocessor),
                                    ('model', lgbm)
                                 ])

    lgbm_pipeline_best_params.fit(X_train, y_train)
    print(lightgbm_search.named_steps['model_lightgbm_RandomizedSearchCV'].best_score_)
    return lgbm_pipeline_best_params

#lgbm_pipeline = LGBMModel(X_train, y_train, preprocessor)

### CatBoost

In [36]:
def CatBoostModel(X_train, y_train, preprocessor):
    
    cb_dflt = CatBoostClassifier(logging_level='Silent')

    param_lst_cb = {
        'n_estimators' : [10],#0, 300, 500, 1000, 1300, 1600, 4000, 7000, 10000, 15000],
        'learning_rate' : [0.0001, 0.001, 0.01, 0.03, 0.1],
        'l2_leaf_reg' : [0.001, 0.01, 0.1],
        'random_strength' : [0.25, 0.5 ,1],
        'max_depth' : [3, 6, 9],
        'min_child_samples' : [2, 5, 10, 15, 20],
        'rsm' : [0.5, 0.7, 0.9]
    }

    catboost = RandomizedSearchCV(estimator = cb_dflt, 
                                  param_distributions = param_lst_cb,
                                  n_iter = 3, 
                                  scoring = 'accuracy', 
                                  n_jobs = 20,
                                  cv = 5)

    cb_pipeline = Pipeline(steps=[
                                    ('preprocessor', preprocessor),
                                    ('model_catboost_RandomizedSearchCV', catboost)
                                 ])

    catboost_search = cb_pipeline.fit(X_train, y_train)

    # CatBoost with tuned hyperparams
    cb = CatBoostClassifier(**catboost_search.named_steps['model_catboost_RandomizedSearchCV'].best_params_, logging_level='Silent')

    # # Bundle preprocessing and modeling code in a pipeline
    cb_pipeline_best_params = Pipeline(steps=[
                                    ('preprocessor', preprocessor),
                                    ('model', cb)
                                 ])

    cb_pipeline_best_params.fit(X_train, y_train)

    print(catboost_search.named_steps['model_catboost_RandomizedSearchCV'].best_score_)
    return cb_pipeline_best_params

#cb_pipeline = CatBoostModel(X_train, y_train, preprocessor)

### Validate best pipe with valid data

In [37]:
def validate_best_pipe(X_valid, y_valid, xgb_pipeline, lgbm_pipeline, cb_pipeline):    
    
    #xgb_pipeline
    if xgb_pipeline is not None:
        preds = xgb_pipeline.predict(X_valid)

    #     score_xgb_mae = mean_absolute_error(y_valid, preds)
    #     rmse_xgb = np.sqrt(mean_squared_error(y_valid, preds))
        score_xgb = xgb_pipeline.score(X_valid, y_valid)
        #cv_xgb = mean_cross_val(xgb_pipeline, x, y)
    
        print('---xgb---')
    #     print('MAE:', score_xgb_mae)
        print('Score:', score_xgb)
    #     print('RMSE:', rmse_xgb)
        #print('CV:', cv_xgb)
    
    #lgbm_pipeline
    if lgbm_pipeline is not None:
        preds = lgbm_pipeline.predict(X_valid)

    #     score_lgbm_mae = mean_absolute_error(y_valid, preds)
    #     rmse_lgbm = np.sqrt(mean_squared_error(y_valid, preds))
        score_lgbm = lgbm_pipeline.score(X_valid, y_valid)
        #cv_lgbm = mean_cross_val(lgbm_pipeline, x, y)

        print('---lgbm---')
    #     print('MAE:', score_lgbm_mae)
        print('Score:', score_lgbm)
    #     print('RMSE:', rmse_xgb)
        #print('CV:', cv_lgbm)
    
    #cb_pipeline
    if cb_pipeline is not None:
        preds = cb_pipeline.predict(X_valid)

        score_cb_mae = mean_absolute_error(y_valid, preds)
        rmse_cb = np.sqrt(mean_squared_error(y_valid, preds))
        score_cb = cb_pipeline.score(X_valid, y_valid)
        #cv_cb = mean_cross_val(cb_pipeline, x, y)

        print('---cb_pipeline---')
    #     print('MAE:', score_cb_mae)
        print('Score:', score_cb)
    #     print('RMSE:', rmse_cb)
        #print('CV:', cv_cb)

## BaseLine 

In [38]:
# X_train,X_valid,y_train,y_valid = train_test_split(X,y,test_size=0.2)

# def score_competition(X_valid, y_valid, xgb_pipeline, lgbm_pipeline, cb_pipeline):
#     validate_best_pipe(X_valid, y_valid, xgb_pipeline, lgbm_pipeline, cb_pipeline)

# score_competition(X_valid, y_valid, xgb_pipeline, lgbm_pipeline, cb_pipeline)

In [39]:
# X.shape

## Redundant features

Now that we have more or less finished analysing our data and gaining insights through the various analysis and visualization, we will have to leverage on these insights to guide our preprocessing decision, so as to provide a clean and error-free data for our model to train on later on.

### Multicollinearity - highly correlated features

From the above correlation matrix, we can pinpointed certain features that are highly correlated that we will remove to avoid the problem of multicollinearity.

In [40]:
# X.drop(['Features 1','Features 2','Features 3','Features 4'], axis=1, inplace=True)

### Features with alot of missing values

Apart from these highly correlated features, we also can remove features that is not very useful in prediction due to many missing values that can provide any useful information.

In [41]:
# plt.figure(figsize=(25,8))
# plt.title('Number of missing rows')
# missing_count = pd.DataFrame(X.isnull().sum(), columns=['sum']).sort_values(by=['sum'],ascending=False).head(20).reset_index()
# missing_count.columns = ['features','sum']
# sns.barplot(x='features',y='sum', data = missing_count)

In [42]:
# X.drop(['','',''], axis=1, inplace=True)

In [43]:
# [X.shape, y.shape]

### Useless features

We can also remove features that does not have any linear relationship with target identified on the Correlation analysis.

In [44]:
# fig,axes = plt.subplots(1,2, figsize=(15,5))
# sns.regplot(x=num_col_TARGET['NH4_5'], y=TARGET, data=num_col_TARGET, ax = axes[0], line_kws={'color':'black'})
# sns.regplot(x=num_col_TARGET['NO2_1'], y=TARGET, data=num_col_TARGET, ax = axes[1], line_kws={'color':'black'})
# fig.tight_layout(pad=2.0)

In [45]:
# correlation[[TARGET]].sort_values([TARGET], ascending=False).tail(10)

In [46]:
# X.drop(['',''], axis=1, inplace=True)

In [47]:
# X.shape

### Removing features that have mostly just 1 value

Earlier during our Univariate analysis, if we found that some features mostly consist of just a single value or 0s, which is not useful to us, we set a user defined threshold at 96% where if a column has more than 96% of the same value, we will render the features to be useless and remove it, since there isnt much information we can extract from

In [48]:
# cat_col = X.select_dtypes(include=['object']).columns
# overfit_cat = []
# for i in cat_col:
#     counts = X[i].value_counts()
#     zeros = counts.iloc[0]
#     if zeros / len(X) * 100 > 96:
#         overfit_cat.append(i)

# overfit_cat = list(overfit_cat)
# print(overfit_cat)

In [49]:
# X = X.drop(overfit_cat, axis=1)

In [50]:
# num_col = X.select_dtypes(exclude=['object']).copy()
# overfit_num = []
# for i in num_col:
#     counts = X[i].value_counts()
#     zeros = counts.iloc[0]
#     if zeros / len(X) * 100 > 96:
#         overfit_num.append(i)

# overfit_num = list(overfit_num)
# print(overfit_num)

In [51]:
# X = X.drop(overfit_num, axis=1)

In [52]:
# print("Categorical Features with >96% of the same value: ",overfit_cat)
# print("Numerical Features with >96% of the same value: ",overfit_num)

### Dealing with duplicated values

In [53]:
# print('Number of duplicated values in dataset: ', X.duplicated().sum())

In [54]:
#X.drop_duplicates(inplace=True)

## Dealing with outliers

Removing outliers will prevent our models performance from being affected by extreme values. From our boxplot earlier, we can pinpointed features with extreme outliers and remove the them based on certain threshold value.

In [55]:
# def preprocess_outliers(data):
#     svm = SGDOneClassSVM(nu=0.55)
#     yhat = svm.fit_predict(data.drop('id', axis=1))
#     mask = yhat != -1
#     data_SVM = data.loc[mask, :].reset_index(drop=True)
   

#     iso = IsolationForest(random_state=0)
#     yhat = iso.fit_predict(data_SVM.drop('id', axis=1))
#     mask = yhat != -1
#     data_ISO = data_SVM.loc[mask, :].reset_index(drop=True)


#     lof = LocalOutlierFactor(n_neighbors=3, contamination=0.1)
#     yhat = lof.fit_predict(data_ISO.drop(['id', 'target'], axis=1))
#     mask = yhat != -1
#     data_final = data_ISO.loc[mask, :].reset_index(drop=True)
  
#     dp = np.random.choice(data_final[data_final[TARGET]==7].index.to_numpy(), 120, replace=False)
#     return data_final.drop(dp)

# X = preprocess_outliers(X)

In [56]:
# X.shape

In [57]:
# out_col = ['','','','','']
# fig = plt.figure(figsize=(20,5))
# for index,col in enumerate(out_col):
#     plt.subplot(1,5,index+1)
#     sns.boxplot(y=col, data=X)
# fig.tight_layout(pad=1.5)

## Filling Missing Values

Machine learning model is unable to deal with missing values, thus we need to deal with them based on our understanding of the features. These missing values are denoted NAN as we have seen earlier during our data exploration.

In [58]:
# pd.DataFrame(X.isnull().sum(), columns=['sum']).sort_values(by=['sum'],ascending=False)

### Ordinal features

We will replace the ordinal missing values with NA, which will be mapped later on when we encode them into an ordered arrangement

In [59]:
# X[ord_col] = X[ord_col].fillna("NA")

### Categorical features

We will replace the missing value of our categorical features with the most frequent occurrence (mode) of the individual features.

In [60]:
# X[cat_col] = X.groupby("CategoricalFeature")[cat_col].transform(lambda x: x.fillna(x.mode()[0]))

### Numerical features

For Numerical Features, the common approach will be to replace the missing value with the mean of the feature distribution. However, certain features have wide variance in their distribution.

In [61]:
# print("Mean of : ", X[''].mean())
# print("Mean of : ", X[''].mean())

In [62]:
# neigh_lot = X.groupby('')[''].mean().reset_index(name='')
# neigh_garage = X.groupby('')[''].mean().reset_index(name='')

# fig, axes = plt.subplots(1,2,figsize=(22,8))
# axes[0].tick_params(axis='x', rotation=90)
# sns.barplot(x='', y='', data=neigh_lot, ax=axes[0])
# axes[1].tick_params(axis='x', rotation=90)
# sns.barplot(x='', y='', data=neigh_garage, ax=axes[1])

In [63]:
# #for correlated relationship
# X[''] = X.groupby('')[''].transform(lambda x: x.fillna(x.mean()))

# #numerical
# X[num_col] = X[num_col] = X[num_col].fillna(X[num_col].mean())

### Mapping ordinal features

May there are some columns which are ordinal by nature, which represents the quality or condition of certain features. We can map the respective strings to a value. The better the quality, the higher the value.

In [64]:
# X.head

In [65]:
# ordinal_map = {'Ex': 5,'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA':0}
# fintype_map = {'GLQ': 6,'ALQ': 5,'BLQ': 4,'Rec': 3,'LwQ': 2,'Unf': 1, 'NA': 0}
# expose_map = {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'NA': 0}
# fence_map = {'GdPrv': 4,'MnPrv': 3,'GdWo': 2, 'MnWw': 1,'NA': 0}

In [66]:
# for col in ord_col:
#     X[col] = X[col].map(ordinal_map)
    
# fin_col = ['','']
# for col in fin_col:
#     X[col] = X[col].map(fintype_map)

# X[''] = X[''].map(expose_map)
# X[''] = X[''].map(fence_map)

In [67]:
# cols = X.select_dtypes(np.number).drop(['id', TARGET], axis=1).columns

# for i in cols:
#     X[i]=RobustScaler().fit_transform(X[i].values.reshape(-1,1))

# transformer = RobustScaler().fit(X[cols])
# X[cols] = transformer.transform(X[cols])

## Remove bad labels

In [68]:
# def remove_bad_lbs(X, bad_lbs):
#     return X[~X['id'].isin(bad_lbs)]

In [69]:
# X = remove_bad_lbs(X,[])

In [70]:
# X.shape

## Set useless features to zero

We can also disable features that does not have any linear relationship with target identified on the Correlation analysis.

In [71]:
# selected_features = []
# for i in X.columns:
#     if i not in selected_features:
#         X[i] = 0

In [72]:
#X.shape

# Feature Engineering <a class="anchor"  id="chapter2"></a>

After removing the outliers, highly correlated features and imputing missing values, we can now proceed with adding additional information for our model to train on. This is done by the means of - Feature Engineering.

## Creating Features

In [73]:
def make_features(df):
    
    periods = 20
    df["anglez"] = abs(df["anglez"])
    df["anglez_diff"] = df.groupby('series_id')['anglez'].diff(periods=periods).fillna(method="bfill").astype('float16')
    df["enmo_diff"] = df.groupby('series_id')['enmo'].diff(periods=periods).fillna(method="bfill").astype('float16')
    df["anglez_rolling_mean"] = df["anglez"].rolling(periods,center=True).mean().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["enmo_rolling_mean"] = df["enmo"].rolling(periods,center=True).mean().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["anglez_rolling_max"] = df["anglez"].rolling(periods,center=True).max().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["enmo_rolling_max"] = df["enmo"].rolling(periods,center=True).max().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["anglez_rolling_std"] = df["anglez"].rolling(periods,center=True).std().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["enmo_rolling_std"] = df["enmo"].rolling(periods,center=True).std().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["anglez_diff_rolling_mean"] = df["anglez_diff"].rolling(periods,center=True).mean().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["enmo_diff_rolling_mean"] = df["enmo_diff"].rolling(periods,center=True).mean().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["anglez_diff_rolling_max"] = df["anglez_diff"].rolling(periods,center=True).max().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["enmo_diff_rolling_max"] = df["enmo_diff"].rolling(periods,center=True).max().fillna(method="bfill").fillna(method="ffill").astype('float16')
    
    return df

features = [
            #"series_id",
            "step",
            "hour",
            "anglez",
            "anglez_rolling_mean",
            "anglez_rolling_max",
            "anglez_rolling_std",
            "anglez_diff",
            "anglez_diff_rolling_mean",
            "anglez_diff_rolling_max",
            "enmo",
            "enmo_rolling_mean",
            "enmo_rolling_max",
            "enmo_rolling_std",
            "enmo_diff",
            "enmo_diff_rolling_mean",
            "enmo_diff_rolling_max",
           ]

In [74]:
# def mathematical_transforms(df):
#     X = pd.DataFrame()
#     #...
#     return X


# def interactions(df):
#     X = pd.get_dummies(df., prefix="")
#     X = X.mul(df., axis=0)
#     return X


# def counts(df):
#     X = pd.DataFrame()
#     X[""] = df[[
#         "",
#         "",
#     ]].gt(0.0).sum(axis=1)
#     return X


# def break_down(df):
#     X = pd.DataFrame()
#     X[""] = df. .str.split("_", n=1, expand=True)[0]
#     return X


# def group_transforms(df):
#     X = pd.DataFrame()
#     X[""] = df.groupby("")[""].transform("median")
#     return X

# def label_encode(df):
#     X = df.copy()
#     for colname in X.select_dtypes(["category"]):
#         X[colname] = X[colname].cat.codes
#     return X

## Skewed target

In [75]:
# # before transforation
# plt.figure(figsize=(10,6))
# plt.title("Before transformation of Class")
# dist = sns.distplot(X[TARGET],norm_hist=False)

Distribution is skewed to the right, where the tail on the curve’s right-hand side is longer than the tail on the left-hand side, and the mean is greater than the mode. This situation is also called positive skewness.
Having a skewed target will affect the overall performance of our machine learning model, thus, one way to alleviate will be to using log transformation on skewed target reducing the skewness of the distribution.

In [76]:
# X[TARGET] = np.log(X[TARGET])

In [77]:
# # after transforation
# plt.figure(figsize=(10,6))
# plt.title("After transformation of Class")
# dist = sns.distplot(np.log(X[TARGET]),norm_hist=False)

## Clustering K-Means or Cluster-distance

Clustering simply means the assigning of data points to groups based upon how similar the points are to each other. A clustering algorithm makes "birds of a feather flock together," so to speak.

When used for feature engineering, we could attempt to discover groups of customers representing a market segment, for instance, or geographic areas that share similar weather patterns. Adding a feature of cluster labels can help machine learning models untangle complicated relationships of space or proximity.

K-means clustering measures similarity using ordinary straight-line distance (Euclidean distance, in other words). It creates clusters by placing a number of points, called centroids, inside the feature-space. Each point in the dataset is assigned to the cluster of whichever centroid it's closest to. The "k" in "k-means" is how many centroids (that is, clusters) it creates. You define the k yourself.

In [78]:
# cluster_features = [
#     "",
#     ""
# ]


# def cluster_labels(df, features, n_clusters=20):
#     X = df.copy()
#     X_scaled = X.loc[:, features]
#     X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)
#     kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=0)
#     X_new = pd.DataFrame()
#     X_new["Cluster"] = kmeans.fit_predict(X_scaled)
#     return X_new


# def cluster_distance(df, features, n_clusters=20):
#     X = df.copy()
#     X_scaled = X.loc[:, features]
#     X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)
#     kmeans = KMeans(n_clusters=20, n_init=50, random_state=0)
#     X_cd = kmeans.fit_transform(X_scaled)
#     # Label features and join to dataset
#     X_cd = pd.DataFrame(
#         X_cd, columns=[f"Centroid_{i}" for i in range(X_cd.shape[1])]
#     )
#     return X_cd

# def plot_cluester(X):
#     Xy = X.copy()
#     Xy["Cluster"] = Xy.Cluster.astype("category")
#     Xy[TARGET] = y
#     sns.relplot(
#         x="value", y=TARGET, hue="Cluster", col="variable",
#         height=4, aspect=1, facet_kws={'sharex': False}, col_wrap=3,
#         data=Xy.melt(
#             value_vars=features, id_vars=[TARGET, "Cluster"],
#         ),
#     );

In [79]:
# plot_cluester(X)

## PCA

There are two ways you could use PCA for feature engineering.

The first way is to use it as a descriptive technique. Since the components tell you about the variation, you could compute the MI scores for the components and see what kind of variation is most predictive of your target. That could give you ideas for kinds of features to create -- a product of 'Height' and 'Diameter' if 'Size' is important, say, or a ratio of 'Height' and 'Diameter' if Shape is important. You could even try clustering on one or more of the high-scoring components.

The second way is to use the components themselves as features. Because the components expose the variational structure of the data directly, they can often be more informative than the original features.

PCA Best Practices
* There are a few things to keep in mind when applying PCA:
* PCA only works with numeric features, like continuous quantities or counts.
* PCA is sensitive to scale. It's good practice to standardize your data before applying PCA, unless you know you have good reason not to.
* Consider removing or constraining outliers, since they can have an undue influence on the results.

Let's choose a few features that are highly correlated with our target.

In [80]:
# def apply_pca(X, standardize=True):
#     # Standardize
#     if standardize:
#         X = (X - X.mean(axis=0)) / X.std(axis=0)
#     # Create principal components
#     pca = PCA()
#     X_pca = pca.fit_transform(X)
#     # Convert to dataframe
#     component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
#     X_pca = pd.DataFrame(X_pca, columns=component_names)
#     # Create loadings
#     loadings = pd.DataFrame(
#         pca.components_.T,  # transpose the matrix of loadings
#         columns=component_names,  # so the columns are the principal components
#         index=X.columns,  # and the rows are the original features
#     )
#     return pca, X_pca, loadings


# def plot_variance(pca, width=8, dpi=100):
#     # Create figure
#     fig, axs = plt.subplots(1, 2)
#     n = pca.n_components_
#     grid = np.arange(1, n + 1)
#     # Explained variance
#     evr = pca.explained_variance_ratio_
#     axs[0].bar(grid, evr)
#     axs[0].set(
#         xlabel="Component", title="% Explained Variance", ylim=(0.0, 1.0)
#     )
#     # Cumulative Variance
#     cv = np.cumsum(evr)
#     axs[1].plot(np.r_[0, grid], np.r_[0, cv], "o-")
#     axs[1].set(
#         xlabel="Component", title="% Cumulative Variance", ylim=(0.0, 1.0)
#     )
#     # Set up figure
#     fig.set(figwidth=8, dpi=100)
#     return axs

In [81]:
# #Let's choose a few features that are highly correlated with our target, SalePrice.
# pca_features = [
#     "Amount"
# ]

# def pca_inspired(df):
#     X = pd.DataFrame()
#     X["Amount"] = df.Amount + df.
#     X["Amount"] = df. * df.
#     return X


# def pca_components(df, pca_features):
#     X = df.loc[:, pca_features]
#     _, X_pca, _ = apply_pca(X)
#     return X_pca

In [82]:
# def corrplot(df, method="pearson", annot=True, **kwargs):
#     sns.clustermap(
#         df.corr(method),
#         vmin=-1.0,
#         vmax=1.0,
#         cmap="icefire",
#         method="complete",
#         annot=annot,
#         **kwargs,
#     )

# corrplot(df_train, annot=None)

In [83]:
# def indicate_outliers(df):
#     X_new = pd.DataFrame()
#     X_new["Outlier"] = (df. == "") & (df. == "")
#     return X_new

## Target Encoding

Target encoding, is instead meant for categorical features. It's a method of encoding categories as numbers, like one-hot or label encoding, with the difference that it also uses the target to create the encoding. This makes it what we call a supervised feature engineering technique.

A target encoding is any kind of encoding that replaces a feature's categories with some number derived from the target.

There is, however, a way you can use target encoding without having to use held-out encoding data. It's basically the same trick used in cross-validation:

1. Split the data into folds, each fold having two splits of the dataset.
1. Train the encoder on one split but transform the values of the other.
1. Repeat for all the splits.

This way, training and transformation always take place on independent sets of data, just like when you use a holdout set but without any data going to waste.

In [84]:
# class CrossFoldEncoder:
#     def __init__(self, encoder, **kwargs):
#         self.encoder_ = encoder
#         self.kwargs_ = kwargs  # keyword arguments for the encoder
#         self.cv_ = KFold(n_splits=5)

#     # Fit an encoder on one split and transform the feature on the
#     # other. Iterating over the splits in all folds gives a complete
#     # transformation. We also now have one trained encoder on each
#     # fold.
#     def fit_transform(self, X, y, cols):
#         self.fitted_encoders_ = []
#         self.cols_ = cols
#         X_encoded = []
#         for idx_encode, idx_train in self.cv_.split(X):
#             fitted_encoder = self.encoder_(cols=cols, **self.kwargs_)
#             fitted_encoder.fit(
#                 X.iloc[idx_encode, :], y.iloc[idx_encode],
#             )
#             X_encoded.append(fitted_encoder.transform(X.iloc[idx_train, :])[cols])
#             self.fitted_encoders_.append(fitted_encoder)
#         X_encoded = pd.concat(X_encoded)
#         X_encoded.columns = [name + "_encoded" for name in X_encoded.columns]
#         return X_encoded

#     # To transform the test data, average the encodings learned from
#     # each fold.
#     def transform(self, X):
#         from functools import reduce

#         X_encoded_list = []
#         for fitted_encoder in self.fitted_encoders_:
#             X_encoded = fitted_encoder.transform(X)
#             X_encoded_list.append(X_encoded[self.cols_])
#         X_encoded = reduce(
#             lambda x, y: x.add(y, fill_value=0), X_encoded_list
#         ) / len(X_encoded_list)
#         X_encoded.columns = [name + "_encoded" for name in X_encoded.columns]
#         return X_encoded

Usage:

In [85]:
# encoder = CrossFoldEncoder(MEstimateEncoder, m=1)
# X_encoded = encoder.fit_transform(X, y, cols=["", ""])

In [86]:
# X_encoded.head()

In [87]:
# X_encoded_t = encoder.transform(test)

In [88]:
# X_encoded_t

In [89]:
# X.head()

In [90]:
# X['Amount'] = X_encoded['Amount_encoded']
# X['Time'] = X_encoded['Time_encoded']

In [91]:
# X = X.join(X_encoded)
# test = test.join(X_encoded_t)

In [92]:
# X.head()

In [93]:
# test

### Scaling Manual or by Pipeline

RobustScaler is a transformation technique that removes the median and scales the data according to the quantile range (defaults to IQR: Interquartile Range). The IQR is the range between the 1st quartile (25th quantile) and the 3rd quartile (75th quantile). It is also robust to outliers, which makes it ideal for data where there are too many outliers that will drastically reduce the number of training data.

Fit RobustScaler on both Train and Test set is a mistake. By fitting the scaler on both train and test set, we exposed ourselves to the problem of Data Leakage. Data Leakage is a problem when information from outside the training dataset is used to create the model. If we fit the scaler on both training and test data, our training data characteristics will contain the distribution of our testset. As such, we are unknowningly passing in information about our test data into the final training data for training, which will not give us the opportunity to truly test our model on data it has never seen before.
Fit the scaler just on training data, and then transforming it on both training and test data

If a distributions are very different for train and test set. This could cause serious problem, because the algorithm will be able to distinct between those sets more easily.

We can convert the features by breaking it down into an hour of the dayfeature or drop it.

In [94]:
# X = X.drop('', axis=1)
# test = test.drop('', axis=1)

In [95]:
# X.head(10)

In [96]:
# cols = X.select_dtypes(np.number).drop(['id', TARGET], axis=1).columns

# transformer = RobustScaler().fit(X[cols])
# X[cols] = transformer.transform(X[cols])

# # #test[cols] = transformer.transform(test[cols])

In [97]:
#cols

In [98]:
# X.head(10)

In [99]:
train = make_features(train)

X = train[features]
y = train['awake']

In [100]:
#train

or pipeline

In [101]:
features_num = train.select_dtypes(exclude=['object']).drop([TARGET], axis=1).copy().columns
features_cat = train.select_dtypes(include=['object']).drop(['series_id'], axis=1).copy().columns 

print(features_num) 
print(features_cat)

transformer_num = make_pipeline(
    SimpleImputer(strategy="constant"), # there are a few missing values
    #StandardScaler(),
    RobustScaler()
)
transformer_cat = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="NA"),
    OneHotEncoder(handle_unknown='ignore'),
)

preprocessor = make_column_transformer(
    (transformer_num, features_num),
    (transformer_cat, features_cat)
)

Index(['step', 'anglez', 'enmo', 'hour', 'anglez_diff', 'enmo_diff',
       'anglez_rolling_mean', 'enmo_rolling_mean', 'anglez_rolling_max',
       'enmo_rolling_max', 'anglez_rolling_std', 'enmo_rolling_std',
       'anglez_diff_rolling_mean', 'enmo_diff_rolling_mean',
       'anglez_diff_rolling_max', 'enmo_diff_rolling_max'],
      dtype='object')
Index([], dtype='object')


In [102]:
# X.shape

In [103]:
#X

In [104]:
# save some memory
del train
gc.collect();

## Predict

In [105]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=2020)

### Randomized search

In [106]:
# save some memory
del X, y
gc.collect();

In [107]:
xgb_pipeline = XGBModel(X_train, y_train, preprocessor)
#lgbm_pipeline = LGBMModel(X_train, y_train, preprocessor)
#cb_pipeline = CatBoostModel(X_train, y_train, preprocessor)

def score_competition(X_valid, y_valid, xgb_pipeline, lgbm_pipeline, cb_pipeline):
    validate_best_pipe(X_valid, y_valid, xgb_pipeline, lgbm_pipeline, cb_pipeline)

score_competition(X_valid, y_valid, xgb_pipeline, None, None)

[01:51:52] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

---xgb---
Score: 0.8456879818193203


In [108]:
# xgb_pipeline

or

### Cross validation

In [109]:
# def mean_cross_val(model, X, y):
#     score = cross_val_score(model, X, y,scoring="accuracy", cv=2)
#     mean = score.mean()
#     return mean

### Basic model

In [110]:
# xgb_pipeline = XGBClassifier(n_estimators=10)
# xgb_pipeline.fit(X_train,y_train)
# # xgm.score(X_valid,y_valid)

In [111]:
# X.head()

In [112]:
# X.shape

In [113]:
# from sklearn.ensemble import RandomForestClassifier
# xgb_pipeline = RandomForestClassifier(n_estimators=1,
#                                     min_samples_leaf=300,
#                                     random_state=42,n_jobs=-1)

# xgb_pipeline.fit(X, y)

In [114]:
# save some memory
del X_train, X_valid, y_train, y_valid
#del X, y
gc.collect();

### Multiclass classification 

In [115]:
test = (pl.scan_parquet("/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet")
          .with_columns(
              (pl.col("timestamp")
                 .str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z")
                 .dt.hour()
                 .alias("hour")
              )
          )
          .drop("timestamp")
          .collect()
          .to_pandas()
)

In [116]:
test = make_features(test)

X_test = test[features]

test["not_awake"] = xgb_pipeline.predict_proba(X_test)[:,0]
test["awake"]     = xgb_pipeline.predict_proba(X_test)[:,1]

# smoothing the predictions
smoothing_length = 2 * 231
test["score"]  = test["awake"].rolling(smoothing_length,center=True).mean().fillna(method="bfill").fillna(method="ffill")
test["smooth"] = test["not_awake"].rolling(smoothing_length,center=True).mean().fillna(method="bfill").fillna(method="ffill")

# re-binarize
test["smooth"] = test["smooth"].round()

def get_event(df):
    lstCV = zip(df.series_id, df.smooth)
    lstPOI = []
    for (c, v), g in groupby(lstCV, lambda cv: (cv[0], cv[1] != 0 and not pd.isnull(cv[1]))):
        llg = sum(1 for item in g)
        if v is False:
            lstPOI.extend([0] * llg)
        else:
            lstPOI.extend(['onset'] + (llg - 2) * [0] + ['wakeup'] if llg > 1 else [0])
    return lstPOI

test["event"] = get_event(test)

### Binary classification 

In [117]:
# test = make_features(test)

# X_test = test[features]

# test["score"] = xgm.predict_proba(X_test)[:,1]

# test["not_awake"] = 1-test["score"]
# # exponential smoothing of the predictions
# test["smooth"] = test["not_awake"].ewm(span = 100).mean()
# # re-binarize
# test["smooth"] = test["smooth"].round()

# test["event"] = get_event(test)

In [118]:
#test

# Save test predictions to file

In [119]:
sample_submission = test.loc[test["event"] != 0][["series_id","step","event","score"]].copy().reset_index(drop=True).reset_index(names="row_id")
sample_submission.to_csv('submission.csv', index=False)

In [120]:
#sample_submission